In [ ]:
"""
technical_indicators.py

- Simple Moving Average (SMA)
- Exponential Moving Average (EMA)
- Relative Strength Index (RSI)
- Bollinger Bands
- MACD
- Stochastic Oscillator
"""

import pandas as pd
import numpy as np

def moving_average(df: pd.DataFrame, window: int = 20, price_col: str = "Adj Close") -> pd.DataFrame:
    df[f"SMA_{window}"] = df[price_col].rolling(window=window).mean()
    return df

def exponential_moving_average(df: pd.DataFrame, span: int = 20, price_col: str = "Adj Close") -> pd.DataFrame:
    df[f"EMA_{span}"] = df[price_col].ewm(span=span, adjust=False).mean()
    return df

def relative_strength_index(df: pd.DataFrame, window: int = 14, price_col: str = "Adj Close") -> pd.DataFrame:
    delta = df[price_col].diff()
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)
    avg_gain = gain.rolling(window=window, min_periods=window).mean()
    avg_loss = loss.rolling(window=window, min_periods=window).mean()
    rs = avg_gain / avg_loss
    df["RSI"] = 100 - (100 / (1 + rs))
    return df

def bollinger_bands(df: pd.DataFrame, window: int = 20, num_std: float = 2.0, price_col: str = "Adj Close") -> pd.DataFrame:
    """
    Bollinger Bands:
    Middle Band = SMA(window)
    Upper Band = Middle Band + num_std * std(window)
    Lower Band = Middle Band - num_std * std(window)
    
    Might change up at a later time to add more 
    """
    sma = df[price_col].rolling(window).mean()
    std = df[price_col].rolling(window).std()
    
    df['Bollinger_Mid'] = sma
    df['Bollinger_Upper'] = sma + num_std * std
    df['Bollinger_Lower'] = sma - num_std * std
    
    return df

def macd(df: pd.DataFrame, fast: int = 12, slow: int = 26, signal: int = 9, price_col: str = "Adj Close") -> pd.DataFrame:
    """
    I've compiled MACD according to the following:
    MACD (Moving Average Convergence Divergence):
    MACD Line = EMA(fast) - EMA(slow)
    Signal Line = EMA(MACD Line, signal)
    Histogram = MACD Line - Signal Line
    """
    ema_fast = df[price_col].ewm(span=fast, adjust=False).mean()
    ema_slow = df[price_col].ewm(span=slow, adjust=False).mean()
    df['MACD_Line'] = ema_fast - ema_slow
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=signal, adjust=False).mean()
    df['MACD_Hist'] = df['MACD_Line'] - df['MACD_Signal']
    return df

def stochastic_oscillator(df: pd.DataFrame, k_window: int = 14, d_window: int = 3):
    
    low_min = df['Low'].rolling(window=k_window).min()
    high_max = df['High'].rolling(window=k_window).max()
    df['%K'] = ((df['Close'] - low_min) / (high_max - low_min)) * 100
    df['%D'] = df['%K'].rolling(window=d_window).mean()
    return df
